<center
style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8 ; border-radius: 0px; font-size:500%; text-align:center;padding:3.0px; background: #ffffff00; border-top: 2px solid #512DA8; border-bottom: 2px solid #512DA8 ; border-left: 2px solid #512DA8; ; border-right: 2px solid #512DA8"> Predicting Loan Payback <img src="https://www.kaggle.com/competitions/91722/images/header" width="400" style="border: 0px solid #512DA8; border-radius: 1px" loading="lazy" />
</center>

<center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius: 0px; font-size:140%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 3px solid #512DA8; border-top: 2px solid k" > Playground Series - Season 5, Episode 11 </center>

# <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 1- Prep the environment </center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
# from ydata_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance, cv
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, SelectKBest, RFE, chi2
from sklearn.model_selection import (train_test_split, GridSearchCV, KFold, RepeatedKFold,
                                     RepeatedStratifiedKFold, RandomizedSearchCV, cross_val_score,
                                     StratifiedKFold)
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler, Normalizer, minmax_scale, 
                                   PowerTransformer, QuantileTransformer, LabelEncoder,
                                   RobustScaler, StandardScaler, FunctionTransformer,
                                   LabelEncoder, OneHotEncoder, OrdinalEncoder)
from sklearn.compose import make_column_transformer

from sklearn.pipeline import make_pipeline

import optuna
from optuna.samplers import TPESampler

from yellowbrick.regressor import ResidualsPlot, PredictionError

import shap

import warnings
warnings.filterwarnings('ignore')

# Set Seaborn theme with dark grid
sns.set_theme(style="white", palette="Set2", font_scale=0.8)

# Update matplotlib parameters
plt.rcParams.update({
    'axes.facecolor': '#222222', 
    'figure.facecolor': '#222222', 
    'text.color': '#FFF9C4',   
    'axes.labelcolor': '#FFF9C4',    
    'xtick.color': '#FFF9C4',      
    'ytick.color': '#FFF9C4',        
    'grid.color': '#444444',         
    'axes.edgecolor': 'white'        
})

# verify the versions
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')
print(f'optuna version : {optuna.__version__}')

# <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 2- Load and Preview the Datasets </center>

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv', index_col='id')
subm = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
orig_raw = pd.read_csv('/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv')

target = 'loan_paid_back'

train.head()

In [ ]:
orig_raw.head()

In [ ]:
orig = orig_raw[train.columns.tolist()]

orig.head()

In [ ]:
train.info()

In [ ]:
num_feats = test.select_dtypes(include='number').columns.tolist()
cat_feats = test.select_dtypes(exclude='number').columns.tolist()

print(f'numeric features: {num_feats}, \n\nCategory features: {cat_feats}')

# <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 3- EDA </center>

In [ ]:
plt.figure(figsize=(15, 4))
for n, feat in enumerate(num_feats, start=1):
    plt.subplot(1, 5, n)
    sns.kdeplot(train, x=feat, hue=target, fill=True)
    plt.suptitle(f'Distribution of numeric features grouped by {target}', fontsize=14, color='white')
    
plt.tight_layout(pad=2, h_pad=2, w_pad=2)

In [ ]:
for cat_feat in cat_feats[:-1]:
    plt.figure(figsize=(15, 3.6))
    for n, feat in enumerate(num_feats, start=1):
        plt.subplot(1, 5, n)
        sns.violinplot(train, x=feat, y=cat_feat)
        if n > 1:
            plt.ylabel('')
            plt.yticks([])
        # plt.title(feat, color='orange', fontsize=8)
    plt.suptitle(f'Distribution of numeric features grouped by {cat_feat}', fontsize=14, color='white')
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(15, 11))
for n, feat in enumerate(num_feats, start=1):
    plt.subplot(2, 3, n)
    sns.violinplot(train, x=feat, y=cat_feats[-1])
    plt.title(feat, color='orange', fontsize=12)
plt.title(f'{feat} by {cat_feat}')
plt.tight_layout()

In [ ]:
# Create the figure and GridSpec layout
fig = plt.figure(figsize=(12, 12))
gs = GridSpec(5, 2, width_ratios=[1, 1])

for c, col in enumerate(cat_feats, start=0):
    if col != 'grade_subgrade':
        ax0 = fig.add_subplot(gs[c, 0])
        train[col].value_counts().plot.barh(color='darkgrey')
        for count in ax0.containers:
            ax0.bar_label(count, label_type='center')
        plt.title(col, fontsize=10)
        plt.ylabel('')

ax1 = fig.add_subplot(gs[:, 1])
train[col].value_counts().plot.barh(color='darkgrey')
plt.title(col, fontsize=10)
for count in ax1.containers:
    ax1.bar_label(count, label_type='center')
plt.ylabel('')
plt.suptitle('Counts of elements within cat_features', fontsize=14, color='white')
plt.tight_layout(pad=1, h_pad=1, w_pad=5)
plt.show()

In [ ]:
target_counts = train[target].value_counts()

fig = plt.figure(figsize=(10, 5))
gs = GridSpec(2, 2, height_ratios=[2, 1], width_ratios=[2, 3])

ax0 = fig.add_subplot(gs[:, :-1])
# ax0 = target_counts.plot.bar(color=['#e86100', '#da1d81'])
ax0 = target_counts.plot.bar()
for count in ax0.containers:
    ax0.bar_label(count, label_type='center', fmt='%d')
ax1 = fig.add_subplot(gs[:, 1:])
ax1 = target_counts.plot.pie(autopct='%.1f%%',
                            shadow = True,
                            radius=1.2,
                            explode=[0.05, 0.1],
                            startangle=270)
ax1 = pd.Series({' ': 1}).plot.pie(colors=['k'], radius=0.38, ax=ax1)
ax1.set_ylabel('')
plt.suptitle('Counts of target classes in train data')
plt.tight_layout()

# <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 4- Prep the Data and target </center>

In [ ]:
train.columns

In [ ]:
# Create a grade feature from grade_subgrade
for df in [train, test]:
    df['grade'] = df['grade_subgrade'].apply(lambda x: x[0])
    df['interest_amount'] = df['loan_amount']*df['interest_rate']/100
    df['int_amnt/ann_income'] = df['annual_income']/df['interest_amount']

# Order of the ordinal features
educ_level_order = ['PhD', "Master's", "Bachelor's", 'High School', 'Other' ]
grade_subgrade_order = sorted(train['grade_subgrade'].unique())#.tolist())
grade_order = sorted(train['grade'].unique().tolist())

custom_categories = [
    educ_level_order,
    grade_subgrade_order,
    grade_order
]

In [ ]:
train.head()

In [ ]:
num_feats = test.select_dtypes(include='number').columns.tolist()
cat_feats = test.select_dtypes(exclude='number').columns.tolist()

print(f'numeric features: {num_feats}, \n\nCategory features: {cat_feats}')

In [ ]:
use_original = False

if use_original:
    train_ = pd.concat([train, orig], ignore_index=True)
else:
    train_ = train

X = train_.copy()
y = X.pop(target)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=64)

[d.shape for d in [X_train, X_valid]]

# <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 5- Model </center>

## <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 5-1 Build and fit the Model </center>

In [ ]:
X.head()

In [ ]:
# Ordinal cat features
ord_cat_feat = ['education_level', 'grade_subgrade', 'grade']
# Nominal cat features
nom_cat_feat = ['gender', 'marital_status', 'employment_status', 'loan_purpose']

In [ ]:
# scaler = MinMaxScaler()
scaler = StandardScaler()
# scaler = RobustScaler()
ohe_enc = OneHotEncoder()
# lab_enc = LabelEncoder()
ord_enc = OrdinalEncoder(categories=custom_categories)

features_trans = make_column_transformer(
    (scaler, num_feats),
    (ohe_enc, nom_cat_feat),
    (ord_enc, ord_cat_feat),
    remainder='drop', 
    sparse_threshold=0
)

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
        "max_depth": trial.suggest_int("max_depth", 2, 20),
        "num_leaves": trial.suggest_int("num_leaves", 4, 128),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 1.0),
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "objective": "regression",
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.5),
        "max_bin": trial.suggest_int("max_bin", 100, 255),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 2.0),
        "importance_type": trial.suggest_categorical("importance_type", ["split", "gain"]),
        "device": "gpu"  # Uncomment if GPU is available
    }

    model = make_pipeline(
        features_trans,
        LGBMClassifier(**params, verbose=-1)
    )

    if cv_scorer:
        # Cross-validation (recommended)
        scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
        return scores.mean()
    else:
        # Alternatively
        model.fit(X_train, y_train)
        
        preds = model.predict_proba(X_valid)[:, 1]
        score = roc_auc_score(y_valid, preds)
        return score

In [ ]:
def Run_Pass_lgbm_study(n_trials=1):
    if n_trials > 1:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials, timeout=72000, show_progress_bar=True)
        best_study_params = study.best_params

        print(f"Number of finished trials: {len(study.trials)}")
        trial = study.best_trial
        print(f"Best trial RMSE score: {trial.value:.6f}")
    else:
        print("No need to run Optuna, we will use the parameters obtained earlier.")
        
        best_study_params = {'n_estimators': 740, 
                             'learning_rate': 0.3672308069747649, 
                             'max_depth': 9, 
                             'num_leaves': 63, 
                             'min_child_samples': 28, 
                             'colsample_bytree': 0.6201179506563272,
                             'subsample': 0.5553467489822561, 
                             'reg_alpha': 0.7488056521301951, 
                             'reg_lambda': 0.8471092109243122, 
                             'boosting_type': 'dart',
                             'min_split_gain': 0.17879954286459038,
                             'max_bin': 255, 
                             'scale_pos_weight': 0.8049250863686583, 
                             'importance_type': 'gain',
                             'verbose': -1
                            }
    
    print(f"\nBest parameters: {best_study_params}")
    return best_study_params

In [ ]:
# Decide how optuna is scored
cv_scorer=False

# Run the optimization
lgbm_best_params = Run_Pass_lgbm_study(n_trials=100)

In [ ]:
lgbm_params = lgbm_best_params

model = make_pipeline(features_trans, LGBMClassifier(**lgbm_params))

model.fit(X_train, y_train)

## <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 5-2 Model Cross-Validation: OOF </center>

In [ ]:
ns=5

splitter = KFold(n_splits=ns, shuffle=True, random_state=84)

for f, (tr_ind, va_ind) in enumerate(splitter.split(X, y), start=1):
    # Split data and target in folds
    X_tr, X_va = X.iloc[tr_ind],  X.iloc[va_ind]
    y_tr, y_va = y.iloc[tr_ind],  y.iloc[va_ind]
    # Define the predictor
    predictor = make_pipeline(features_trans,  LGBMClassifier(**lgbm_params))
    # Fit the predictor
    predictor.fit(X_tr, y_tr)
    # Predict n the validation data
    y_va_proba = predictor.predict_proba(X_va)[:, 1]
    # Score the predictions
    score = roc_auc_score(y_va, y_va_proba)
    # False vs True predictions rates
    fpr, tpr, _ = roc_curve(y_va, y_va_proba)
    plt.plot(fpr, tpr, label='Fold_{} auc_score: {:.6f}'.format(f, score))
    plt.plot([0,1], [0,1], linestyle='--', color='green')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
plt.show()
   

## <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 5-3 Model Prediction on test data </center>

In [ ]:
pred_proba = model.predict_proba(test)

pred_proba

## <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #512DA8; border-radius:20px; font-size:120%; text-align:center;padding:3.0px; background: #FFF9C4; border-left: 20px solid #512DA8; border-right: 20px solid #512DA8; border-bottom: 2px solid #512DA8; border-top: 2px solid #512DA8" > 5-4 Prep the Submission File </center>

In [ ]:
subm[target] = pred_proba[:, 1]

subm.head()

In [ ]:
subm.to_csv('submission.csv', index=False)

print('\033[92mThe file is ready for submission\033[0m')